In [9]:
import os
os.environ ["JAVA_HOME"] = "/opt/homebrew/Cellar/openjdk/21.0.2"
os.environ ["SPARK_HOME"] = "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec"

In [10]:
import findspark
findspark.init('/opt/homebrew/Cellar/apache-spark/3.5.1/libexec')

In [11]:
spark_url = "local"

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master(spark_url).appName("DE3").config("spark.ui.port", "4040").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/22 18:23:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
sc = spark.sparkContext

In [14]:
sc

<SparkContext master=local appName=DE3>

# Import data and preprocess the data

In [15]:
data_path = './appstore_games.csv'
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [16]:
df.count()

17007

### Drop NA

In [17]:
df = df.na.drop()

In [18]:
df

DataFrame[URL: string, ID: int, Name: string, Subtitle: string, Icon URL: string, Average User Rating: string, User Rating Count: string, Price: string, In-app Purchases: string, Description: string, Developer: string, Age Rating: string, Languages: string, Size: string, Primary Genre: string, Genres: string, Original Release Date: string, Current Version Release Date: string]

### Change format

In [19]:
df = df.withColumn("Average User Rating", df["Average User Rating"].cast("double"))
df = df.withColumn("Price", df["Price"].cast("double"))

df = df.withColumn("User Rating Count", df["User Rating Count"].cast("int"))
df = df.withColumn("Size", df["Size"].cast("int"))

In [20]:
df

DataFrame[URL: string, ID: int, Name: string, Subtitle: string, Icon URL: string, Average User Rating: double, User Rating Count: int, Price: double, In-app Purchases: string, Description: string, Developer: string, Age Rating: string, Languages: string, Size: int, Primary Genre: string, Genres: string, Original Release Date: string, Current Version Release Date: string]

## How many row and column

In [21]:
print(f"Row: {df.count()}")
print(f"Column: {len(df.columns)}")

Row: 1937
Column: 18


24/04/22 18:24:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Preview top 6 rows

In [22]:
df.show(6)

+--------------------+---------+--------------------+--------------------+--------------------+-------------------+-----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+---------------------+----------------------------+
|                 URL|       ID|                Name|            Subtitle|            Icon URL|Average User Rating|User Rating Count|Price|    In-app Purchases|         Description|           Developer|          Age Rating|           Languages|    Size|       Primary Genre|              Genres|Original Release Date|Current Version Release Date|
+--------------------+---------+--------------------+--------------------+--------------------+-------------------+-----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-----------

## What is the average size of all games

In [23]:
from pyspark.sql.functions import avg

avg_size_of_games = df.agg(avg("Size").alias("Average Size of Games"))

avg_size_of_games.show()

+---------------------+
|Average Size of Games|
+---------------------+
| 2.1334109757801765E8|
+---------------------+



## How much does it cost for the game with the most expensive price

In [24]:
from pyspark.sql.functions import max

most_expensive_price = df.agg(max("Price").alias("Most expensive price"))

most_expensive_price.show()

+--------------------+
|Most expensive price|
+--------------------+
|              1639.0|
+--------------------+



## Based on the average of ‘Average User Rating’, is it true that paid games score much better than free games by large margin (> 1)?

In [25]:
from pyspark.sql.functions import avg, col

avg_user_rating_free = df.filter(df.Price == 0.0).agg(avg("Average User Rating").alias("Average User Rating"))
avg_user_rating_paid = df.filter(df.Price > 0).agg(avg("Average User Rating").alias("Average User Rating"))

avg_user_rating_paid = avg_user_rating_paid.first()['Average User Rating']
avg_user_rating_free = avg_user_rating_free.first()['Average User Rating']

print(f"{avg_user_rating_paid}")
print(f"{avg_user_rating_free}")
print(avg_user_rating_paid > avg_user_rating_free + 1)

4.229007633587786
4.3129855715871255
False


## Based on the average of ‘User Rating Count’, is it true that free games have more rating counts than paid games by large margin (> 1000)?

In [26]:
from pyspark.sql.functions import avg, col

avg_user_rating_count_free = df.filter(df.Price == 0.0).agg(avg("User Rating Count").alias("User Rating Count"))
avg_user_rating_count_paid = df.filter(df.Price > 0).agg(avg("User Rating Count").alias("User Rating Count"))

avg_user_rating_count_paid = avg_user_rating_count_paid.first()['User Rating Count']
avg_user_rating_count_free = avg_user_rating_count_free.first()['User Rating Count']

print(f"{avg_user_rating_count_paid}")
print(f"{avg_user_rating_count_free}")
print(avg_user_rating_count_free > avg_user_rating_count_paid + 1000)

5023.22962962963
9496.05049944506
True


24/04/23 00:10:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1034530 ms exceeds timeout 120000 ms
24/04/23 00:10:03 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/23 00:10:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$